# Loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
# load all csv files
data_1 = "princeton_20240101_20251015.csv"
data_2 = "princeton_20230601_20231231.csv"
data_3 = "princeton_lots_20220501_20230531_FINAL.csv"
data_4 = "princeton.csv" # everything pre 5/1/2022

# read all csv files
df1 = pd.read_csv(data_1)
df2 = pd.read_csv(data_2)
df3 = pd.read_csv(data_3)
df4 = pd.read_csv(data_4)

df = pd.concat([df1, df2, df3, df4], ignore_index=True)

/tmp/ipykernel_2354554/456078567.py:10: DtypeWarning: Columns (0,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv(data_3)
/tmp/ipykernel_2354554/456078567.py:11: DtypeWarning: Columns (7,8,13,15,17,20,21,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv(data_4)


# Cleaning datatypes

In [3]:
df_cleaned = df.copy()

In [6]:
# prefer strings for IDs
for col in ['raw_auction_id', 'artist_id', 'raw_lot_id', 'auction_num']:
    df_cleaned[col] = (
        df_cleaned[col]
        .astype('string')
        .str.replace(r'\.0$', '', regex=True)
        .str.strip()
    )

In [4]:
# clean auction date
df_cleaned['auction_start_date'] = pd.to_datetime(
    df_cleaned['auction_start_date'],
    format='%Y-%m-%d',    
    errors='coerce'        
)

In [8]:
# clean prices
price_cols = [
    'price_usd_zeroied',
    'price_estimate_min_usd_zeroied',
    'price_estimate_max_usd_zeroied',
    'price_estimate_min',
    'price_estimate_max',
    'price_sold',
]

for c in price_cols:
    s = pd.to_numeric(df_cleaned[c], errors='coerce')
    df_cleaned[c] = s.round().astype('Int64') 

In [9]:
# clean bought_in as boolean
df_cleaned['bought_in'] = df_cleaned['price_sold'].isna().astype('boolean')

In [10]:
# change datatypes to integers
year_cols = [
    'artwork_creation_year',
    'artist_birth',
    'lot_num'
]

for c in year_cols:
    s = pd.to_numeric(df_cleaned[c], errors='coerce')
    df_cleaned[c] = s.round().astype('Int64')

In [11]:
# change remaining columns to strings
obj_cols = df_cleaned.select_dtypes(include=['object']).columns

df_cleaned[obj_cols] = df_cleaned[obj_cols].astype('string')

# Consolidating data

In [12]:
# drop rows without price estimates
df_cleaned = df_cleaned.dropna(subset=["price_estimate_min", "price_estimate_max"])

In [13]:
# cleaning medium
df_cleaned["medium_final"] = (
    df_cleaned["medium_final"]
    .str.strip()   
    .str.lower()  
)

df_cleaned["medium_final"] = df_cleaned["medium_final"].replace({
    "painting": "paintings",
    "others": "other",
})

In [14]:
# cleaning tail of auction houses

# specific auction house fixes
mask = df_cleaned["auction_house_name"] == "Phillips de Pury & Company"
df_cleaned.loc[mask, "auction_house_name"] = "Phillips"

mask = df_cleaned["auction_house_name"] == "Bonhams & Butterfields"
df_cleaned.loc[mask, "auction_house_name"] = "Bonhams"

mask = df_cleaned["auction_house_name"] == "Poly Auction"
df_cleaned.loc[mask, "auction_house_name"] = "Poly International Auction"

mask = df_cleaned["auction_house_name"] == "Bonhams & Brooks"
df_cleaned.loc[mask, "auction_house_name"] = "Bonhams"

# keep houses until we hit 95% of all lots
counts = df_cleaned["auction_house_name"].value_counts()
cumshare = counts.cumsum() / counts.sum()
major_houses = cumshare[cumshare <= 0.95].index 

df_cleaned["auction_house_grouped"] = df_cleaned["auction_house_name"].where(
    df_cleaned["auction_house_name"].isin(major_houses),
    "Other auction house",
)

In [15]:
# cleaning tail of auction locations

# specific city fixes
mask = df_cleaned["auction_location"] == "Baron Ribeyre & Associes, E. Farrando SVV, Paris"
df_cleaned.loc[mask, "auction_location"] = "Paris"

mask = df_cleaned["auction_location"] == "Via Pitteri, Milan"
df_cleaned.loc[mask, "auction_location"] = "Milan"

mask = df_cleaned["auction_location"] == "Poulain, F. Tajan, Paris"
df_cleaned.loc[mask, "auction_location"] = "Paris"

mask = df_cleaned["auction_location"] == "Paris - Drouot"
df_cleaned.loc[mask, "auction_location"] = "Paris"

mask = df_cleaned["auction_location"] == "Berlin-Grunewald"
df_cleaned.loc[mask, "auction_location"] = "Berlin"

mask = df_cleaned["auction_location"] == "Inc, Altadena"
df_cleaned.loc[mask, "auction_location"] = "Altadena"

mask = df_cleaned["auction_location"] == "Royere et Lajeunesse, Versailles"
df_cleaned.loc[mask, "auction_location"] = "Versailles"

mask = df_cleaned["auction_location"] == "Palais Dorotheum Wien"
df_cleaned.loc[mask, "auction_location"] = "Vienna"

mask = df_cleaned["auction_location"] == "Berlin-Charlottenburg"
df_cleaned.loc[mask, "auction_location"] = "Berlin"

mask = df_cleaned["auction_location"] == "Antwerp-Berchem"
df_cleaned.loc[mask, "auction_location"] = "Antwerp"

mask = df_cleaned["auction_location"] == "London, New Bond Street"
df_cleaned.loc[mask, "auction_location"] = "London"

mask = df_cleaned["auction_location"] == "London, Knightsbridge"
df_cleaned.loc[mask, "auction_location"] = "London"

mask = df_cleaned["auction_location"] == "Inc., New Orleans"
df_cleaned.loc[mask, "auction_location"] = "New Orleans"

mask = df_cleaned["auction_location"] == "Via Pontaccio, Milan"
df_cleaned.loc[mask, "auction_location"] = "Milan"

mask = df_cleaned["auction_location"] == "Inc., Dowington"
df_cleaned.loc[mask, "auction_location"] = "Dowington"

mask = df_cleaned["auction_location"] == "Inc., New York"
df_cleaned.loc[mask, "auction_location"] = "New York"

mask = df_cleaned["auction_location"] == "Ltd., Mystic"
df_cleaned.loc[mask, "auction_location"] = "Mystic"

mask = df_cleaned["auction_location"] == "Online, New York"
df_cleaned.loc[mask, "auction_location"] = "New York"

mask = df_cleaned["auction_location"] == "Lyon Brotteaux"
df_cleaned.loc[mask, "auction_location"] = "Lyon"

mask = df_cleaned["auction_location"] == "Inc., Columbia"
df_cleaned.loc[mask, "auction_location"] = "Columbia"

# keep locations until we hit 95% of all lots
counts = df_cleaned["auction_location"].value_counts()
cumshare = counts.cumsum() / counts.sum()
major_locations = cumshare[cumshare <= 0.95].index 

df_cleaned["auction_location_grouped"] = df_cleaned["auction_location"].where(
    df_cleaned["auction_location"].isin(major_locations),
    "Other auction location",
)

In [16]:
pd.set_option("display.max_columns", None)

In [17]:
df_cleaned

,raw_lot_id,auction_house_name,raw_auction_id,auction_location,auction_name,auction_num,auction_start_date,lot_image_url,lot_num,price_estimate_min,price_estimate_max,price_sold,bought_in,price_kind,currency,artwork_condition_in,artwork_creation_year,artwork_description,artwork_edition_current,artwork_edition_size,artwork_exhibited,artwork_literature,artwork_markings,artwork_materials,artwork_measurements_width,artwork_measurements_height,artwork_measurements_depth,artwork_measurements_unit,artwork_name,artwork_provenance,artwork_size_notes,catalog_notes,medium_final,price_usd_zeroied,price_estimate_min_usd_zeroied,price_estimate_max_usd_zeroied,artist_name,artist_nationality,artist_birth,artist_id,auction_house_grouped,auction_location_grouped
0,298153205,Sotheby's,506709,New York,A Scholar Collects,N11437,2024-01-31,https://s3-us-west-2.amazonaws.com/ciqcrawler/...,10,12000,16000,25400,False,premium,USD,"The sheet is laid down on an old backing, thou...",1826,Pastel on blue paper; inscribed and dated by t...,NaN,NaN,"Paris, Grand Palais, Élisabeth Louise Vigée Le...","N. Jeffares, ""Élisabeth-Louise Vigée Le Brun,""...",inscribed and dated by the artist on a sheet o...,Pastel on blue paper,17.7,14.200,NaN,centimeters,Landscape of the Ardennes with the Church of M...,The artist's estate; Caroline de Rivière (1793...,<NA>,<NA>,works on paper,25400,12000,16000,Elisabeth-Louise Vigée Le Brun,French,1755,<NA>,Sotheby's,New York
1,298153206,Sotheby's,506709,New York,A Scholar Collects,N11437,2024-01-31,https://s3-us-west-2.amazonaws.com/ciqcrawler/...,9,12000,16000,44450,False,premium,USD,"The sheet is laid down on an old backing, thou...",1821,Pastel on blue paper inscribed and dated by th...,NaN,NaN,"Paris, Grand Palais, Élisabeth Louise Vigée Le...","N. Jeffares, ""Élisabeth-Louise Vigée Le Brun,""...",inscribed and dated by the artist on a sheet o...,Pastel on blue paper,20.9,15.000,NaN,centimeters,Sky study,The artist's estate; Caroline de Rivière (1793...,<NA>,"Among the most celebrated European portrait, h...",works on paper,44450,12000,16000,Elisabeth-Louise Vigée Le Brun,French,1755,<NA>,Sotheby's,New York
2,298153207,Sotheby's,506709,New York,A Scholar Collects,N11437,2024-01-31,https://s3-us-west-2.amazonaws.com/ciqcrawler/...,20,60000,80000,190500,False,premium,USD,Please note that this pastel has not been view...,<NA>,Pastel bears inscription on a label to the str...,NaN,NaN,"New York, The Metropolitan Museum of Art, Past...","B. Sani, “Tra disegno e pittura: il pastello c...",<NA>,Pastel,27.4,32.000,NaN,centimeters,"A young woman wearing pearl earrings, with flo...","Possibly Duke of Saint Albans, Portland Place;...",<NA>,<NA>,works on paper,190500,60000,80000,Rosalba Carriera,Italian,1675,<NA>,Sotheby's,New York
3,298153212,Sotheby's,506709,New York,A Scholar Collects,N11437,2024-01-31,https://s3-us-west-2.amazonaws.com/ciqcrawler/...,11,700000,1000000,3085000,False,premium,USD,"This portrait by Elizabeth Vigee-LeBrun, the l...",<NA>,Pastel bears inscription in black ink on the o...,NaN,NaN,"New York, Wildenstein, The Winds of Revolution...","Élisabeth Louise Vigée Le Brun, Mémoires d’une...",<NA>,"Pastel, bears inscription in black ink on the ...",38.1,50.165,NaN,centimeters,Self-Portrait In Traveling Costume,Presented by the artist in Rome to François Gu...,<NA>,<NA>,works on paper,3085000,700000,1000000,Elisabeth-Louise Vigée Le Brun,French,1755,<NA>,Sotheby's,New York
4,298153213,Sotheby's,506709,New York,A Scholar Collects,N11437,2024-01-31,https://s3-us-west-2.amazonaws.com/ciqcrawler/...,8,120000,180000,304800,False,premium,USD,Hinge mounted to a modern decorative mount. A ...,<NA>,Black and white chalk with stumping over trace...,NaN,NaN,"Paris, Galeries Nationales du Grand Palais; Ne...","J. Baillio and X. Salmon, Élisabeth Louise Vig...",<NA>,Black and white chalk with stumping over trace...,28.2,43.200,NaN,centimeters,Profile Portrait of the Duchesse de Polignac,"Private collection; Anonymous sale, London

# Export data

In [20]:
# export df file
df_cleaned.to_parquet("df_cleaned.parquet", index=False)